<a href="https://colab.research.google.com/github/fred-lafleche/Exchange/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#imports
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plot
from sklearn.utils import resample
from scipy.sparse import coo_matrix
from sklearn.utils import resample
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
shoppersInput_file= "/content/drive/MyDrive/Colab Notebooks/online_shoppers_intention.csv"
heartInput_file = "/content/drive/MyDrive/Colab Notebooks/heart.csv"
marketingInput_file = "/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv"

shopDataPd = pd.read_csv(shoppersInput_file)
heartDataPd = pd.read_csv(heartInput_file)
marketDataPd = pd.read_csv(marketingInput_file)
shopDataPd.head()
heartDataPd.head()
marketDataPd.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


Online Shoppers Intention

In [6]:
#From Assignment 1

# Normalization
dataArray = shopDataPd.values
numericData = dataArray[:,[0,1,2,3,4,5,6,7,8,9]]
scaler = preprocessing.StandardScaler().fit(numericData)
scalledNumericData = scaler.transform(numericData)

# One hot encoding
categoricalData = dataArray[:,[10,15,16]]
categoricalLabel = dataArray[:,[17]]
encoder1 = preprocessing.OneHotEncoder()
encoder2 = preprocessing.OneHotEncoder()
encoder1.fit(categoricalData)
encoder2.fit(categoricalLabel)
categoricalData=encoder1.transform(categoricalData).toarray()
categoricalLabel = encoder2.transform(categoricalLabel).toarray()

#join data back together
data = np.column_stack((scalledNumericData, categoricalData))
#add back columns 11 to 14 and 17
data = np.column_stack((data, dataArray[:,[11,12,13,14]]))
label = categoricalLabel[:,[1]]

Heart Data

In [7]:
heartDataArray = heartDataPd.values
print(heartDataArray.size)
heartData = np.delete(heartDataArray, 13, 1)
print(heartData.size)
heartLabel = heartDataArray[:,[13]]
print(heartLabel.size)

4242
3939
303


Marketing Campaign

In [82]:
marketDataArray = marketDataPd.values
print(marketDataArray.size)
marketLabel = marketDataArray[:,[6]] #label
marketData = np.delete(marketDataArray, 6, 1) #remove label
marketData = np.delete(marketData, 6, 1) #date
marketData = np.delete(marketData, 3, 1) #Marital Status
marketData = np.delete(marketData, 2, 1) #Education

# Normalization
scalerMarket = preprocessing.StandardScaler().fit(marketData)
scalledMarketData = scalerMarket.transform(marketData)

# One hot encoding
categoricalMarketData = marketDataArray[:,[2,3]]
encoder = preprocessing.OneHotEncoder()
encoder.fit(categoricalMarketData)
categoricalMarketData=encoder.transform(categoricalMarketData).toarray()

encoder = preprocessing.OneHotEncoder()
encoder.fit(marketLabel)
marketLabel=encoder.transform(marketLabel).toarray()
marketLabel = marketLabel[:,[0]] #1 for absence

#join data back together
marketData = np.column_stack((scalledMarketData, categoricalMarketData))

print(marketData[100])
print(marketLabel[0])
print(marketLabel[1])
print(marketLabel[2])
print(marketData.size)

print(marketLabel.size)

64264
[ 1.66304719 -0.73608511  1.46263636 -0.82303932 -1.071551    0.44153476
  4.21375442 -0.27202076 -0.1212581  -0.12246811 -0.01863288 -0.68817587
  0.33379569  0.45418246 -0.55414289 -1.3687842  -0.28177297 -0.28270484
 -0.28083892 -0.26166148 -0.11714818 -0.09781206  0.          0.
 -0.42052997  0.          0.          0.          1.          0.
  0.          0.          0.          1.          0.          0.
  0.          0.        ]
[1.]
[0.]
[1.]
84208
2216


# Single Decision Tree

In [83]:
#Heart data
SDT = tree.DecisionTreeClassifier()
SDTscores = cross_val_score(SDT,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (SDTscores.mean(), SDTscores.std() * 2))

#Marketing data
SDT2 = tree.DecisionTreeClassifier()
SDTscores2 = cross_val_score(SDT2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (SDTscores2.mean(), SDTscores2.std() * 2))

#Online Shoppers data
SDT3 = tree.DecisionTreeClassifier()
SDTscores3 = cross_val_score(SDT3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (SDTscores3.mean(), SDTscores3.std() * 2))


Heart Data Accuracy: 0.77 (+/- 0.14)
Marketing Data Accuracy: 0.86 (+/- 0.04)
Online Shoppers Data Accuracy: 0.85 (+/- 0.08)


# Random Forest

In [84]:
#Heart data
RF = RandomForestClassifier()
RFscores = cross_val_score(RF,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (RFscores.mean(), RFscores.std() * 2))

#Marketing data
RF2 = RandomForestClassifier()
RFscores2 = cross_val_score(RF2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (RFscores2.mean(), RFscores2.std() * 2))

#Online Shoppers data
RF3 = RandomForestClassifier()
RFscores3 = cross_val_score(RF3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (RFscores3.mean(), RFscores3.std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

Heart Data Accuracy: 0.83 (+/- 0.13)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

Marketing Data Accuracy: 0.90 (+/- 0.04)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

Online Shoppers Data Accuracy: 0.89 (+/- 0.03)


# Support Vector Machine

In [85]:
#Heart data
s_v_m = svm.SVC()
SVMscores = cross_val_score(s_v_m,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (SVMscores.mean(), SVMscores.std() * 2))

#Marketing data
s_v_m2 = svm.SVC()
SVMscores2 = cross_val_score(s_v_m2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (SVMscores2.mean(), SVMscores2.std() * 2))

#Online Shoppers data
s_v_m3 = svm.SVC()
SVMscores3 = cross_val_score(s_v_m3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (SVMscores3.mean(), SVMscores3.std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Heart Data Accuracy: 0.66 (+/- 0.16)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Marketing Data Accuracy: 0.89 (+/- 0.03)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Online Shoppers Data Accuracy: 0.89 (+/- 0.03)


# K-Nearest Neighbor

In [86]:
#Heart data
knn = KNeighborsClassifier()
KNNscores = cross_val_score(knn,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (KNNscores.mean(), KNNscores.std() * 2))

#Marketing data
knn2 = KNeighborsClassifier()
KNNscores2 = cross_val_score(knn2,marketData,marketLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (KNNscores2.mean(), KNNscores2.std() * 2))

#Online Shoppers data
knn3 = KNeighborsClassifier()
KNNscores3 = cross_val_score(knn3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (KNNscores3.mean(), KNNscores3.std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

Heart Data Accuracy: 0.65 (+/- 0.16)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

Heart Data Accuracy: 0.80 (+/- 0.05)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

Online Shoppers Data Accuracy: 0.86 (+/- 0.03)


# Multi-layer Perceptron

In [87]:
#Heart data
MLP = MLPClassifier(random_state=1, max_iter=300)
MLPscores = cross_val_score(MLP,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (MLPscores.mean(), MLPscores.std() * 2))

#Marketing data
MLP2 = MLPClassifier(random_state=1, max_iter=300)
MLPscores2 = cross_val_score(MLP2,marketData,marketLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (MLPscores2.mean(), MLPscores2.std() * 2))

#Online Shoppers data
MLP3 = MLPClassifier(random_state=1, max_iter=300)
MLPscores3 = cross_val_score(MLP3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (MLPscores3 .mean(), MLPscores3 .std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d

Heart Data Accuracy: 0.81 (+/- 0.13)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimizati

Heart Data Accuracy: 0.90 (+/- 0.03)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Online Shoppers Data Accuracy: 0.88 (+/- 0.05)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Gradient Boosting

In [89]:
#Heart data
GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
GBscores = cross_val_score(GB,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (GBscores.mean(), GBscores.std() * 2))

#Marketing data
GB2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
GBscores2 = cross_val_score(GB2,marketData,marketLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (GBscores2.mean(), GBscores2.std() * 2))

#Online Shoppers data
GB3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
GBscores3 = cross_val_score(GB3,data,label,cv=10)
print("Online Shoppers Data Accuracy: %0.2f (+/- %0.2f)" % (GBscores3 .mean(), GBscores3 .std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

Heart Data Accuracy: 0.80 (+/- 0.14)


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

Heart Data Accuracy: 0.89 (+/- 0.04)


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

Online Shoppers Data Accuracy: 0.88 (+/- 0.04)


# Results

In [99]:
print(SDTscores3)
print(RFscores3)
print(SVMscores3)
print(KNNscores3)
print(MLPscores3)
print(GBscores3)

[0.87591241 0.90186537 0.90510949 0.87429035 0.86780211 0.78588808
 0.79886456 0.82643958 0.82887267 0.80454177]
[0.87104623 0.89943228 0.92457421 0.88402271 0.89862125 0.88158962
 0.8702352  0.88807786 0.8945661  0.87753447]
[0.8945661  0.90186537 0.91403082 0.88158962 0.88969992 0.87915653
 0.87429035 0.87185726 0.87753447 0.86618005]
[0.88888889 0.87510138 0.88564477 0.87591241 0.87104623 0.8540146
 0.85077048 0.85969181 0.85077048 0.8296837 ]
[0.87185726 0.90510949 0.92619627 0.88077859 0.88726683 0.85077048
 0.84833739 0.86861314 0.87104623 0.84752636]
[0.84509327 0.87510138 0.90673155 0.88077859 0.90186537 0.84022709
 0.87266829 0.87996756 0.87591241 0.87266829]


In [97]:
print ("Single Decision Tree: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (SDTscores.mean(),SDTscores2.mean(),SDTscores3.mean()))
print ("Random Forest: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (RFscores.mean(),RFscores2.mean(),RFscores3.mean()))
print ("Support Vector Machine: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (SVMscores.mean(),SVMscores2.mean(),SVMscores3.mean()))
print ("K-nearest Neighbor: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (KNNscores.mean(),KNNscores2.mean(),KNNscores3.mean()))
print ("Multi-layer Perceptron: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (MLPscores.mean(),MLPscores2.mean(),MLPscores3.mean()))
print ("Gradient Boosting: \nHeart: %0.2f \nMarketing: %0.2f \nShoppers: %0.2f \n" % (GBscores.mean(),GBscores2.mean(),GBscores3.mean()))


Single Decision Tree: 
Heart: 0.77 
Marketing: 0.86 
Shoppers: 0.85 

Random Forest: 
Heart: 0.83 
Marketing: 0.90 
Shoppers: 0.89 

Support Vector Machine: 
Heart: 0.66 
Marketing: 0.89 
Shoppers: 0.89 

K-nearest Neighbor: 
Heart: 0.65 
Marketing: 0.80 
Shoppers: 0.86 

Multi-layer Perceptron: 
Heart: 0.81 
Marketing: 0.90 
Shoppers: 0.88 

Gradient Boosting: 
Heart: 0.80 
Marketing: 0.89 
Shoppers: 0.88 

